In [68]:
import json 

import asyncio

from igvf_async_client import AsyncIgvfApi

import requests

api = AsyncIgvfApi()

In [ ]:
async def get_total(item_type):
    total = (await api.search(type=[item_type], limit=0)).total
    return (item_type, total)

In [ ]:
import time
s = time.time()
res = await asyncio.gather(*(get_total(item_type) for item_type in item_types))
e = time.time()
print(e - s, dict(sorted(res, key=lambda x: -x[1])))

In [97]:
await api.search(
    type=['FileSet']
)
print()

In [22]:
base_url = 'https://api.data.igvf.org'

In [14]:
files_url = (
    'https://api.data.igvf.org/search/'
    '?type=File&file_set.data_use_limitation_summaries=HMB-MDS'
    '&file_set.controlled_access=true'
    '&field=file_set.data_use_limitation_summaries'
    '&field=file_set.controlled_access'
    '&field=file_set.@id'
    '&limit=all'
)

In [99]:
files_url = (
    'https://api.data.igvf.org/search/'
    '?type=File&file_set.data_use_limitation_summaries=GRU'
    '&file_set.controlled_access=true'
    '&field=file_set.data_use_limitation_summaries'
    '&field=file_set.controlled_access'
    '&field=file_set.@id'
    '&limit=all'
    '&file_set.@id=/analysis-sets/IGVFDS0793KZKS/'
)

In [100]:
files = requests.get(files_url).json()['@graph']
files

[{'@id': '/tabular-files/IGVFFI1928OJPU/',
  '@type': ['TabularFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['GRU'],
   '@id': '/analysis-sets/IGVFDS0793KZKS/',
   'controlled_access': True}},
 {'@id': '/index-files/IGVFFI9775WVZK/',
  '@type': ['IndexFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['GRU'],
   '@id': '/analysis-sets/IGVFDS0793KZKS/',
   'controlled_access': True}},
 {'@id': '/matrix-files/IGVFFI7332UUZE/',
  '@type': ['MatrixFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['GRU'],
   '@id': '/analysis-sets/IGVFDS0793KZKS/',
   'controlled_access': True}},
 {'@id': '/tabular-files/IGVFFI1257TMRU/',
  '@type': ['TabularFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['GRU'],
   '@id': '/analysis-sets/IGVFDS0793KZKS/',
   'controlled_access': True}},
 {'@id': '/tabular-files/IGVFFI7212WXZR/',
  '@type': ['TabularFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_sum

In [101]:
files_seen = set()
files_local = {}

samples_seen = set()
samples_local = {}

donors_seen = set()
donors_local = {}

file_sets_seen = set()
file_sets_local = {}

for f in files:
    files_local[f['@id']] = f
    files_seen.add(f['@id'])

file_sets = {f['file_set']['@id'] for f in files}

print(file_sets, len(file_sets), len(files))

{'/analysis-sets/IGVFDS0793KZKS/'} 1 5


In [102]:
while file_sets:
    fs = file_sets.pop()
    if fs in file_sets_seen:
        continue
    file_sets_seen.add(fs)
    full_fs = requests.get(base_url + fs + '@@object').json()
    file_sets_local[fs] = full_fs
    if 'input_file_sets' in full_fs:
        for ifs in full_fs['input_file_sets']:
            print('Found input file set')
            if ifs not in file_sets_seen:
                file_sets.add(ifs)
    if 'files' in full_fs:
        for f in full_fs['files']:
            if f not in files_seen:
                full_file = requests.get(base_url + f + '@@object').json()
                files_local[f] = full_file
                files_seen.add(f)
    if 'samples' in full_fs:
        print('GETTING SAMPLES')
        samples = requests.get(base_url + f'/search/?type=Sample&file_sets.@id={fs}&frame=object&limit=all').json()['@graph']
        for sample in samples:
            samples_seen.add(sample['@id'])
            samples_local[sample['@id']] = sample
    if 'donors' in full_fs:
        print('Getting donors')
        for donor in full_fs['donors']:
            if donor not in donors_seen:
                donors_seen.add(donor)
                print('getting donor', donor)
                full_donor = requests.get(base_url + donor + '@@object').json()
                donors_local[donor] = full_donor

GETTING SAMPLES
Getting donors
getting donor /human-donors/IGVFDO1264SGCS/
getting donor /human-donors/IGVFDO0938OYMI/
getting donor /human-donors/IGVFDO1718HCES/
GETTING SAMPLES
Getting donors
GETTING SAMPLES
Getting donors
GETTING SAMPLES
Getting donors


In [103]:
len(file_sets_local)

4

In [104]:
print('files local', len(files_local))
print('file sets local', len(file_sets_local))
print('samples local', len(samples_local))
print('donors local', len(donors_local))

files local 110
file sets local 4
samples local 13
donors local 3


In [105]:
def print_summary(files_seen, file_sets_seen, samples_seen, donors_seen):
    print(
        json.dumps(
            {
                'files': len(list(sorted(files_seen))),
                'file_sets': len(list(sorted(file_sets_seen))),
                'samples': len(list(sorted(samples_seen))),
                'donors': len(list(sorted(donors_seen))),
            },
            indent=4
        )
    )
    print(
        json.dumps(
            {
                'files': list(sorted(files_seen)),
                'file_sets': list(sorted(file_sets_seen)),
                'samples': list(sorted(samples_seen)),
                'donors': list(sorted(donors_seen)),
            },
            indent=4
        )
    )

In [106]:
print_summary(files_seen, file_sets_seen, samples_seen, donors_seen)

{
    "files": 110,
    "file_sets": 4,
    "samples": 13,
    "donors": 3
}
{
    "files": [
        "/configuration-files/IGVFFI0147QMYN/",
        "/configuration-files/IGVFFI1513BFDT/",
        "/configuration-files/IGVFFI2368QXMP/",
        "/configuration-files/IGVFFI2618ZGAS/",
        "/configuration-files/IGVFFI2891IZHI/",
        "/configuration-files/IGVFFI3611FLZB/",
        "/configuration-files/IGVFFI4613CIJO/",
        "/configuration-files/IGVFFI4678GDCD/",
        "/configuration-files/IGVFFI4864HCIP/",
        "/configuration-files/IGVFFI5716CRTP/",
        "/configuration-files/IGVFFI6057HKMR/",
        "/configuration-files/IGVFFI6152BDFZ/",
        "/configuration-files/IGVFFI7692LKWW/",
        "/configuration-files/IGVFFI7823EBGU/",
        "/configuration-files/IGVFFI8294ZONF/",
        "/configuration-files/IGVFFI8994UTFE/",
        "/configuration-files/IGVFFI9273RDVJ/",
        "/configuration-files/IGVFFI9392EWHS/",
        "/configuration-files/IGVFFI9421YM

In [24]:
a = set()
a.add('x')
a.add('y')
a

{'x', 'y'}

In [56]:
file_sets_local['/measurement-sets/IGVFDS2534CJYA/']['samples']

['/in-vitro-systems/IGVFSM7625PGWV/']

In [17]:
files

[{'@id': '/matrix-files/IGVFFI8142SBEI/',
  '@type': ['MatrixFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['HMB-MDS'],
   '@id': '/analysis-sets/IGVFDS0718WXPI/',
   'controlled_access': True}},
 {'@id': '/tabular-files/IGVFFI9764GKVR/',
  '@type': ['TabularFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['HMB-MDS'],
   '@id': '/analysis-sets/IGVFDS0718WXPI/',
   'controlled_access': True}},
 {'@id': '/configuration-files/IGVFFI7971ZCIE/',
  '@type': ['ConfigurationFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['HMB-MDS'],
   '@id': '/measurement-sets/IGVFDS2259DWKL/',
   'controlled_access': True}},
 {'@id': '/configuration-files/IGVFFI6594MPVX/',
  '@type': ['ConfigurationFile', 'File', 'Item'],
  'file_set': {'data_use_limitation_summaries': ['HMB-MDS'],
   '@id': '/measurement-sets/IGVFDS2534CJYA/',
   'controlled_access': True}},
 {'@id': '/sequence-files/IGVFFI8905WBYU/',
  '@type': ['SequenceFile', 'Fil